# Import the library

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

Import the dataset

In [3]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

# Encoding catagorical data

Label Encoding Gender



In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot encoding geography

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(),[1])],remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [6]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Split the dataset into test and train set

In [7]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 0)

Feature scaling

In [8]:
from sklearn.preprocessing import StandardScaler


# scaling all the columns in the data as it is nessary in ANN
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Building the ANN

Initialize the ANN

In [9]:
ann = tf.keras.models.Sequential() # this initializes the layers of the ANN

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-03-03 23:27:36.880013: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-03 23:27:36.880405: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Adding the input layer

In [10]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Add a second layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Add a output layer

In [12]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Training the ANN

Compiling the ANN

In [13]:
ann.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy']) # catagorical_cross entroy as loss for non binary outputs

Training the ANN

In [14]:
ann.fit(X_train , y_train , batch_size = 32 , epochs = 100)

Epoch 1/100


2023-03-03 23:27:45.817075: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-03 23:27:46.299653: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 2s 4ms/step - loss: 0.6219 - accuracy: 0.6566
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4506 - accuracy: 0.8001
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4345 - accuracy: 0.8055
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4258 - accuracy: 0.8138
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4182 - accuracy: 0.8200
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4094 - accuracy: 0.8285
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3990 - accuracy: 0.8353
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3874 - accuracy: 0.8418
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3772 - accuracy: 0.8465
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3698 - accuracy: 0.8496
E

Predict outputs

In [15]:
print(ann.predict(sc.transform([[1.0,0.0,0.0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

2023-03-03 23:29:36.025427: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 458ms/step
[[False]]


Predict on test case values

In [16]:
y_pred = ann.predict(X_test)

63/63 [==============================] - 0s 2ms/step


In [17]:
y_pred = (y_pred > 0.5)

Create a confusion matrix

In [18]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_pred)
print(cm)

[[1505   90]
 [ 193  212]]


In [19]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

0.8585